# Zomato Recommendation System

A recommendation system is a type of information filtering system that predicts or suggests items or actions a user might be interested in based on their preferences, behavior, or past interactions. These systems are commonly used in various online platforms to help users discover relevant products, services, content, or people.

Recommendation systems are widely used in e-commerce platforms, streaming services, social networks, and other online applications to enhance user experience, increase engagement, and drive revenue by suggesting relevant content or products to users.

# Types of  Recommendation System

1.Content-Based Filtering: This method recommends items similar to those the user has liked or interacted with in the past. It analyzes the attributes or features of items and recommends items with similar characteristics.

2.Collaborative Filtering: Collaborative filtering recommends items based on the preferences of other users. It identifies users with similar preferences or behaviors and suggests items that these similar users have liked or interacted with.

3.Hybrid Recommendation Systems: These systems combine multiple recommendation techniques to provide more accurate and diverse recommendations. For example, a hybrid system might combine content-based filtering with collaborative filtering to take advantage of the strengths of both approaches.

4.Knowledge-Based Recommendation Systems: Knowledge-based recommendation systems use explicit knowledge about users' preferences and domain knowledge about items to generate recommendations. They often involve rules or constraints based on domain expertise.

5.Context-Aware Recommendation Systems: Context-aware recommendation systems consider additional contextual information, such as time, location, or device, to provide more personalized recommendations that are relevant to the user's current situation.

# In this Notebook we are using Content-Based Filtering to suggests similar restaurants based on cosine similarity scores.
 
 Some key  point for this project which will help you to understand futher steps 
 
1.Cosine Similarity: The function calculates cosine similarity scores between restaurants based on certain features (likely cuisine, mean rating, and cost). Cosine similarity is a technique commonly used in content-based recommendation systems to measure the similarity between items based on their feature vectors.
    
2.Feature-based Recommendations: The function identifies restaurants with similar feature vectors to the input restaurant (specified by the name parameter) and recommends them based on their similarity scores. This is a characteristic of content-based recommendation systems, which recommend items similar to those that a user has liked or interacted with in the past, based on the attributes or features of the items.

3.No User Data Considered: The function doesn't explicitly consider user preferences or behaviors. Instead, it relies solely on the features of the restaurants to make recommendations. This aligns with the content-based filtering approach, which doesn't require information about other users' preferences or behaviors.

# Import neccessary Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#reading the dataset and loading datasets 
zomato_real=pd.read_csv("D:\Projects\Zomato\zomato.csv")
zomato_real.head() 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [3]:
zomato_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [4]:
zomato_real.describe()

,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000


Data Cleaning

In [5]:
#Deleting Unnnecessary Columns
zomato=zomato_real.drop(['url','dish_liked','phone'],axis=1)
#Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"

In [6]:
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [7]:
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)
zomato.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   address                      43499 non-null  object
 1   name                         43499 non-null  object
 2   online_order                 43499 non-null  object
 3   book_table                   43499 non-null  object
 4   rate                         43499 non-null  object
 5   votes                        43499 non-null  int64 
 6   location                     43499 non-null  object
 7   rest_type                    43499 non-null  object
 8   cuisines                     43499 non-null  object
 9   approx_cost(for two people)  43499 non-null  object
 10  reviews_list                 43499 non-null  object
 11  menu_item                    43499 non-null  object
 12  listed_in(type)              43499 non-null  object
 13  listed_in(city)              43

In [8]:
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'approx_cost(for two people)',
       'reviews_list', 'menu_item', 'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [9]:
#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city','rate':'rating'})
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rating', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')

In [10]:
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float) # Changing the cost to Float
zomato.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       43499 non-null  object 
 1   name          43499 non-null  object 
 2   online_order  43499 non-null  object 
 3   book_table    43499 non-null  object 
 4   rating        43499 non-null  object 
 5   votes         43499 non-null  int64  
 6   location      43499 non-null  object 
 7   rest_type     43499 non-null  object 
 8   cuisines      43499 non-null  object 
 9   cost          43499 non-null  float64
 10  reviews_list  43499 non-null  object 
 11  menu_item     43499 non-null  object 
 12  type          43499 non-null  object 
 13  city          43499 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 5.0+ MB


In [11]:
#Reading Rate of dataset
zomato['rating'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [12]:
# Removing '/5' from Ratings
zomato = zomato.loc[zomato.rating != 'NEW']
zomato = zomato.loc[zomato.rating != '-'].reset_index(drop=True)

# Convert all values in 'rating' column to strings
zomato['rating'] = zomato['rating'].astype(str)

# Remove '/5' from ratings
zomato['rating'] = zomato['rating'].str.replace('/5', '')

# Strip any leading or trailing whitespace
zomato['rating'] = zomato['rating'].str.strip()

# Convert cleaned ratings to float
zomato['rating'] = zomato['rating'].astype(float)

zomato['rating'].head()


0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rating, dtype: float64

In [13]:
# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)
zomato.cost.unique()

array([800.  , 300.  , 600.  , 700.  , 550.  , 500.  , 450.  , 650.  ,
       400.  , 900.  , 200.  , 750.  , 150.  , 850.  , 100.  ,   1.2 ,
       350.  , 250.  , 950.  ,   1.  ,   1.5 ,   1.3 , 199.  ,   1.1 ,
         1.6 , 230.  , 130.  ,   1.7 ,   1.35,   2.2 ,   1.4 ,   2.  ,
         1.8 ,   1.9 , 180.  , 330.  ,   2.5 ,   2.1 ,   3.  ,   2.8 ,
         3.4 ,  50.  ,  40.  ,   1.25,   3.5 ,   4.  ,   2.4 ,   2.6 ,
         1.45,  70.  ,   3.2 , 240.  ,   6.  ,   1.05,   2.3 ,   4.1 ,
       120.  ,   5.  ,   3.7 ,   1.65,   2.7 ,   4.5 ,  80.  ])

In [14]:
zomato.head(10)

,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
5,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,True,False,3.8,286,Basavanagudi,Casual Dining,North Indian,600.0,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari
6,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,False,False,3.6,8,Mysore Road,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[],Buffet,Banashankari
7,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,True,True,4.6,2556,Banashankari,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,"[('Rated 5.0', 'RATED\n I personally really l...",[],Cafes,Banashankari
8,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,True,False,4.0,324,Banashankari,Cafe,"Cafe, Italian, Continental",700.0,"[('Rated 3.0', ""RATED\n I had been to this pl...",[],Cafes,Banashankari
9,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,True,False,4.2,504,Banashankari,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[],Cafes,Banashankari


In [15]:
zomato['city'].unique()

array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [16]:
zomato.head(10)

,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
5,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,True,False,3.8,286,Basavanagudi,Casual Dining,North Indian,600.0,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari
6,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,False,False,3.6,8,Mysore Road,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[],Buffet,Banashankari
7,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,True,True,4.6,2556,Banashankari,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,"[('Rated 5.0', 'RATED\n I personally really l...",[],Cafes,Banashankari
8,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,True,False,4.0,324,Banashankari,Cafe,"Cafe, Italian, Continental",700.0,"[('Rated 3.0', ""RATED\n I had been to this pl...",[],Cafes,Banashankari
9,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,True,False,4.2,504,Banashankari,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[],Cafes,Banashankari


In [17]:
## Checking Null values
zomato.isnull().sum()

address         0
name            0
online_order    0
book_table      0
rating          0
votes           0
location        0
rest_type       0
cuisines        0
cost            0
reviews_list    0
menu_item       0
type            0
city            0
dtype: int64

In [18]:
## Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rating'][zomato['name'] == restaurants[i]].mean()

In [19]:
zomato.head(10)


,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.100000
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.800000
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700000
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.800000
5,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,True,False,3.8,286,Basavanagudi,Casual Dining,North Indian,600.0,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari,3.800000
6,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,False,False,3.6,8,Mysore Road,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[],Buffet,Banashankari,3.600000
7,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,True,True,4.6,2556,Banashankari,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,"[('Rated 5.0', 'RATED\n I personally really l...",[],Cafes,Banashankari,4.410588
8,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,True,False,4.0,324,Banashankari,Cafe,"Cafe, Italian, Continental",700.0,"[('Rated 3.0', ""RATED\n I had been to this pl...",[],Cafes,Banashankari,4.000000
9,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,True,False,4.2,504,Banashankari,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[],Cafes,Banashankari,4.200000


In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (1,5))

zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

zomato.sample(10)

,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
15371,"Brigade Homestead, Ashoka Pillar, Jayanagar, B...",Cafe Coffee Day,True,False,2.9,33,Jayanagar,Cafe,Cafe,650.0,"[('Rated 2.0', 'RATED\n CCD is known to be th...",[],Cafes,Jayanagar,2.88
30207,"Ground Floor, Brand Factory, 12th Cross, Wilso...",Meat And Eat,True,False,3.2,4,Wilson Garden,Quick Bites,"Fast Food, Burger",500.0,"[('Rated 5.0', 'RATED\n good several'), ('Rat...",[],Delivery,Lavelle Road,2.98
4794,"303, 3rd Floor, 5th Avenue Mall, Brigade Road,...",Tempura Restaurant,False,False,3.9,122,Brigade Road,Casual Dining,"Japanese, Chinese, Tibetan, Momos",500.0,"[('Rated 4.0', ""RATED\n One of the few places...",[],Dine-out,Brigade Road,3.71
16375,"70/71, Seetharam Complex, Bilekahalli, Bannerg...",Status,False,False,3.3,67,Bannerghatta Road,Casual Dining,"North Indian, Chinese",600.0,"[('Rated 5.0', ""RATED\n Tasty pocket friendly...",[],Delivery,Jayanagar,3.08
22100,"Eat Street 11, 80 Feet Road, Opposite Indian O...",The Breakfast Truck,False,False,3.9,64,Koramangala 6th Block,Quick Bites,Continental,550.0,"[('Rated 4.0', ""RATED\n A Sunday afternoon, I...",[],Delivery,Koramangala 4th Block,3.60
29571,"2nd cross, Behind Anjaneya Temple, Opposite Pr...",Body Temple,True,False,3.3,12,BTM,Quick Bites,Healthy Food,500.0,"[('Rated 4.0', ""RATED\n I ordered a keto chic...",[],Dine-out,Koramangala 7th Block,2.94
13190,"1018, 80 Feet Road, Near Wipro Tech Park, 1st ...",Sharma'S Malai & Masala,True,False,4.0,457,Koramangala 1st Block,Casual Dining,North Indian,550.0,"[('Rated 3.0', 'RATED\n Ordered Tandoori Chic...",[],Delivery,HSR,3.84
17906,"13, 7th Cross, 2nd Phase, JP Nagar, Bangalore",Chulha Chauki Da Dhaba,True,False,4.3,2420,JP Nagar,Casual Dining,North Indian,800.0,"[('Rated 5.0', ""RATED\n The Ambiance gives th...",[],Delivery,JP Nagar,4.07
6438,"590-10/1, First Cross, BEML Layout, Tubarahall...",High Sky,False,False,3.6,32,Whitefield,Casual Dining,"Continental, North Indian, Chinese, Arabian",1.0,"[('Rated 4.0', 'RATED\n Limited options in th...",[],Dine-out,Brookefield,2.98
12017,"842/A, 100 Feet Road, Indiranagar, Bangalore",Byblos,True,True,4.2,1347,Indiranagar,Casual Dining,"Lebanese, Arabian, Middle Eastern, Salad, Beve...",1.7,"[('Rated 2.0', ""RATED\n Food and ambiance was...",[],Delivery,Frazer Town,4.10


In [21]:
# Calculate the average rating
average_rating = zomato['Mean Rating'].mean()

# Print the average rating
print("Average Rating:", average_rating)


Average Rating: 3.4541908965249655


# Text Preprocessing

Some of the common text preprocessing / cleaning steps are:

1.Lower casing

2.Removal of Punctuations

3.Removal of Stopwords

4.Removal of URLs

5.Spelling correction

In [22]:
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
25200,"[('Rated 5.0', ""RATED\n The paneer biryani se...","North Indian, Biryani"
21853,"[('Rated 4.0', 'RATED\n The only place In ban...","Cafe, Bakery"
3559,"[('Rated 1.0', 'RATED\n The worst chopsey whi...",Chinese
16528,"[('Rated 1.0', 'RATED\n test west'), ('Rated ...","North Indian, Biryani"
34857,"[('Rated 4.0', 'RATED\n Love the banana leaf ...","Coffee, South Indian"


In [23]:
zomato.head(10)

,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.58
5,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,True,False,3.8,286,Basavanagudi,Casual Dining,North Indian,600.0,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari,3.58
6,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,False,False,3.6,8,Mysore Road,Casual Dining,"North Indian, South Indian, Andhra, Chinese",800.0,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[],Buffet,Banashankari,3.32
7,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,True,True,4.6,2556,Banashankari,"Casual Dining, Cafe","Pizza, Cafe, Italian",600.0,"[('Rated 5.0', 'RATED\n I personally really l...",[],Cafes,Banashankari,4.37
8,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,True,False,4.0,324,Banashankari,Cafe,"Cafe, Italian, Continental",700.0,"[('Rated 3.0', ""RATED\n I had been to this pl...",[],Cafes,Banashankari,3.84
9,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,True,False,4.2,504,Banashankari,Cafe,"Cafe, Mexican, Italian, Momos, Beverages",550.0,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[],Cafes,Banashankari,4.10


In [24]:
## Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
20273,"[('rated 4.0', 'rated\n compact menu yet good...","American, Burger, Momos, Bengali"
28586,"[('rated 3.0', ""rated\n not at all surprising...",Biryani
24801,"[('rated 4.0', 'rated\n hey cb ?\ni ordered t...",Chinese
38315,"[('rated 4.0', ""rated\n looks like there's a ...",Desserts
6037,"[('rated 4.0', 'rated\n decent place for a di...",North Indian


In [25]:
# Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))
zomato[['reviews_list', 'cuisines']].sample(10)


,reviews_list,cuisines
26661,rated 40 ratedn this has been a long pending ...,"Ice Cream, Desserts"
19678,rated 30 ratedn budget chinese food i would s...,Chinese
2389,rated 40 ratedn good and small place to hango...,"Asian, Continental"
19057,rated 45 ratedn good food here tried swarma c...,"Arabian, Mediterranean, North Indian"
30699,rated 30 ratedn not exactly a fan of japanese...,Japanese
17923,rated 30 ratedn the thing which i hate about ...,Andhra
11830,rated 50 ratedn this place absolutely deserve...,"Seafood, South Indian, Bar Food, Mangalorean, ..."
34560,rated 50 ratedn this is a little holeinthewal...,"North Indian, Mughlai"
32940,rated 50 ratedn just now i order from this pl...,"North Indian, Oriya, Chinese"
4787,rated 30 ratedn on one hand loved the petfrie...,"Cafe, American, Italian, Burger"


In [26]:
## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

In [27]:
## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))


In [28]:
zomato[['reviews_list', 'cuisines']].sample(10)

,reviews_list,cuisines
31252,rated 50 ratedn main land china number times b...,Chinese
27292,rated 40 ratedn food good specially pan fried ...,"Continental, Italian"
20900,rated 50 ratedn love seating open spacious air...,"Continental, North Indian"
32508,rated 20 ratedn bad tasteni ordered many times...,"Biryani, North Indian"
18901,rated 50 ratedn woah woah woah never seen big ...,"Cafe, Fast Food"
23414,rated 30 ratedn ordered shahi paneer kalmi kab...,"North Indian, Biryani, Chinese"
2721,rated 50 ratedn orderedn1 chicken mayo wrapn1 ...,"North Indian, Biryani, Fast Food"
35668,rated 40 ratedn seemed quite popular quite cro...,"Cafe, Continental, Momos, Italian, North Indian"
1011,rated 40 ratedn place bookmarks since quite ti...,"Street Food, Sandwich"
30011,rated 30 ratedn small neat little cafe opposit...,"Pizza, Italian, Cafe, Burger, Finger Food, Fas..."


In [29]:
 #RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
restaurant_names

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point'

In [30]:
def get_top_words(column, top_nu_of_words, nu_of_word):
    
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    
    bag_of_words = vec.fit_transform(column)
    
    sum_words = bag_of_words.sum(axis=0)
    
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    return words_freq[:top_nu_of_words]
zomato.head()

,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,[],Buffet,Banashankari,3.58


In [31]:
zomato.sample(10)

,address,name,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
26944,"7, 80 Feet Road, Koramangala 4th Block, Bangalore",Hanoi - Vietnamese Cuisine,True,True,4.4,1811,Koramangala 4th Block,Casual Dining,Vietnamese,1.0,rated 50 ratedn hanoi capital vietnam name sug...,[],Dine-out,Koramangala 6th Block,4.35
11826,"2nd Floor, UB City, Vittal Mallya Road, Lavell...",Amande Patisserie,True,False,4.2,231,Lavelle Road,"Bakery, Dessert Parlor","Bakery, French, Desserts",300.0,rated 40 ratedn nblueberry belgium dark chocol...,"['Mango Cheese Cake Pastry', 'Orange Tart', 'F...",Delivery,Frazer Town,4.02
30098,"73/3, Railway Parallel Road, Kumara Park West,...",Mama Mia!,True,False,3.9,111,Seshadripuram,Dessert Parlor,"Desserts, Cafe",400.0,rated 40 ratedn recently tried gelatos satin s...,[],Delivery,Lavelle Road,4.04
2478,"433/3/1, 4th Floor, 28th Cross, 10th Main, 4th...",Cafe Cosy,False,True,4.1,102,Jayanagar,Cafe,"Cafe, Chinese, Continental, Fast Food",800.0,rated 50 ratedn ambience great hookah flavour ...,[],Delivery,Basavanagudi,3.99
26510,"80, 4th B Cross, Opposite BDA Complex, Koraman...",Daily Bread,False,False,2.8,53,Koramangala 4th Block,"Bakery, Quick Bites","Bakery, Fast Food, Italian",600.0,rated 10 ratedn ordered eggless creamless nutl...,[],Delivery,Koramangala 6th Block,2.51
35943,"Chowdeswari Temple Street, Byrasandra, CV Rama...",Mother'S Meal,True,False,3.8,134,CV Raman Nagar,Quick Bites,North Indian,200.0,rated 30 ratedn ordered dal makhani n disappoi...,[],Delivery,Old Airport Road,3.58
36709,"Pyramid Food Court, Embassy Golf Links, Domlur...",Sizzling Tandoor,False,False,3.1,5,Domlur,Food Court,"Lebanese, Mughlai, North Indian",500.0,rated 20 ratedn located inside egl limited acc...,[],Dine-out,Old Airport Road,2.68
33337,"4, Opposite Garden Layout, Shirdi Sai Nagar, K...",Prajwal The Restaurant,True,False,3.5,47,Marathahalli,Quick Bites,"North Indian, Chinese, Beverages",300.0,rated 50 ratedn place heaven ãx83ãx83ãx82ã...,[],Dine-out,Marathahalli,3.06
13425,"78, Ground Floor, Beside BDA Complex, Sector 6...",Howrah Bridge,True,False,3.3,292,HSR,Casual Dining,"Bengali, Seafood",600.0,rated 10 ratedn shop shut fraud looting custom...,[],Dine-out,HSR,2.91
29183,"62, RB Fashion House, Jyoti Nivas College Road...",Shanmukha,True,False,4.0,788,Koramangala 5th Block,Casual Dining,"Biryani, Andhra, North Indian, Chinese",800.0,rated 50 ratedn little sceptical trying food u...,"['Veg Fried Rice', 'Curd Rice', 'Executive Pac...",Dine-out,Koramangala 7th Block,3.81


In [32]:
zomato.shape

(41237, 15)

In [33]:
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rating', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city', 'Mean Rating'],
      dtype='object')

In [34]:
import pandas

# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

In [35]:
df_percent.shape

(20618, 15)

# Term Frequency-Inverse Document Frequency¶
Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document. This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each column represents a restaurant, as before.

TF-IDF is the statistical method of evaluating the significance of a word in a given document.

TF — Term frequency(tf) refers to how many times a given term appears in a document.

IDF — Inverse document frequency(idf) measures the weight of the word in the document, i.e if the word is common or rare in the entire document. The TF-IDF intuition follows that the terms that appear frequently in a document are less important than terms that rarely appear. Fortunately, scikit-learn gives you a built-in TfIdfVectorizer class that produces the TF-IDF matrix quite easily.

In [36]:
df_percent.set_index('name', inplace=True)

In [37]:
indices = pd.Series(df_percent.index)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Adjust parameters to reduce the size of the TF-IDF matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=5, max_features=5000, stop_words='english')

# Fit and transform the data
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])


In [40]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming tfidf_matrix_reduced is already computed
cosine_similarities = cosine_similarity(tfidf_matrix_reduced, dense_output=False)


In [41]:
from sklearn.decomposition import TruncatedSVD

# Choose the number of components for TruncatedSVD
n_components = 100

# Initialize TruncatedSVD
svd = TruncatedSVD(n_components=n_components)

# Fit TruncatedSVD to the TF-IDF matrix and transform the matrix
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)

# Compute cosine similarities using the reduced TF-IDF matrix
cosine_similarities = cosine_similarity(tfidf_matrix_reduced, dense_output=False)


In [42]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new

In [43]:
# HERE IS A RANDOM RESTAURANT. LET'S SEE THE DETAILS ABOUT THIS RESTAURANT:
df_percent[df_percent.index == 'Pai Vihar'].head()

,address,online_order,book_table,rating,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
name,,,,,,,,,,,,,,
Pai Vihar,"16/A, Ground Floor, Kkmp Building, Vasanth Nag...",True,False,2.8,54,Vasanth Nagar,Quick Bites,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 20 ratedn nice place vasanthnagar extrem...,"['2 Rice Idli', 'Uddina Vada', 'Open Butter Ma...",Delivery,Brigade Road,2.48
Pai Vihar,"1106, OTC Road, Opposite City Corporation Offi...",False,False,3.3,42,City Market,Quick Bites,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 20 ratedn food dry bland dont understand...,[],Dine-out,Residency Road,2.48
Pai Vihar,"16/A, Ground Floor, Kkmp Building, Vasanth Nag...",True,False,2.8,55,Vasanth Nagar,Quick Bites,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,"['Open Butter Masala Dosa', 'Paneer Butter Mas...",Delivery,Malleshwaram,2.48
Pai Vihar,"16/A, Ground Floor, Kkmp Building, Vasanth Nag...",True,False,2.8,55,Vasanth Nagar,Quick Bites,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate hereãx83ãx83ãx82ãx...,[],Delivery,MG Road,2.48
Pai Vihar,"16/A, Ground Floor, Kkmp Building, Vasanth Nag...",True,False,2.8,56,Vasanth Nagar,Quick Bites,"South Indian, Street Food, Chinese, Fast Food",400.0,rated 30 ratedn 12 rate herenneven though tast...,[],Dine-out,Frazer Town,2.48


In [44]:
recommend('Pai Vihar')

TOP 10 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Haji Ali Juice Centre,"Fast Food, Pizza, Rolls, Beverages, Juices",3.82,350.0
Haji Ali Juice Centre,"Beverages, Cafe, Juices, Fast Food",3.82,350.0
The Krishna Grand,"North Indian, South Indian, Chinese, Fast Food...",3.69,750.0
Udupi Ruchi,"South Indian, North Indian, Chinese, Street Food",3.64,200.0
Udupi Nalapaka,"South Indian, North Indian, Chinese",3.61,400.0
Sri Ganesh Juice Junction,"Beverages, Juices, Fast Food",3.56,150.0
Juicemaker,"Beverages, Fast Food",3.56,400.0
Sri Krishna Bhavan,"South Indian, North Indian",3.13,250.0
Juice Lounge,"Beverages, Healthy Food",2.68,250.0
Udupi Delux,"South Indian, North Indian, Chinese",2.03,400.0


In [46]:
# Group data by restaurant type and calculate average rating for each type
average_ratings_by_type = zomato.groupby('rest_type')['Mean Rating'].mean()

# Sort the average ratings in descending order
average_ratings_by_type_sorted = average_ratings_by_type.sort_values(ascending=False)

# Print the restaurant types with the highest average ratings
print("Restaurant types with higher average ratings:")
print(average_ratings_by_type_sorted)


Restaurant types with higher average ratings:
rest_type
Pub, Cafe                    4.650000
Bar, Pub                     4.610000
Microbrewery                 4.457143
Pub, Microbrewery            4.418553
Microbrewery, Pub            4.415238
                               ...   
Dhaba                        2.877059
Food Court, Casual Dining    2.854000
Bakery, Beverage Shop        2.810000
Bakery, Food Court           2.785000
Food Court, Beverage Shop    2.440000
Name: Mean Rating, Length: 87, dtype: float64
